In [9]:
import cv2
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import scipy.misc
import glob
#%matplotlib notebook
#%matplotlib qt
%matplotlib inline


#WARPED PERSPECIVE TRANSFORM
def perspect_transform(img, src, dst):
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))
    return warped

#dst_size = 5 
#bottom_offset = 6
#source = np.float32([[15, 140], [301 ,140],[200, 96], [118, 96]])
#destination = np.float32([[image_imread.shape[1]/2 - dst_size, image_imread.shape[0] - bottom_offset],
#                  [image_imread.shape[1]/2 + dst_size, image_imread.shape[0] - bottom_offset],
#                  [image_imread.shape[1]/2 + dst_size, image_imread.shape[0] - 2*dst_size - bottom_offset], 
#                  [image_imread.shape[1]/2 - dst_size, image_imread.shape[0] - 2*dst_size - bottom_offset],
#                  ])

#WARPED NAIGABLE AND OBSTACLE
#warped = perspect_transform(image_otsu, source, destination)
#WARPED GOLD ROCKS/SAMPLES
#warped_rock = perspect_transform(image_cv2, source, destination)


#GROUND
def navigable_thresh(img):
    blur = cv2.GaussianBlur(img,(5,5),0)
    ret2,threshed_ground = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return threshed_ground

#navigable_world = navigable_thresh(warped)

#OBSTACLES
def obstacle_thresh(img):
    blur = cv2.GaussianBlur(img,(5,5),0)
    ret2,threshed_obstacle = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    return threshed_obstacle

#obstacle_world = obstacle_thresh(warped)

#GOLD ROCKS
#warped_rock = perspect_transform(image_cv2, source, destination)
def rock_thresh(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([4,100,100])
    upper_yellow = np.array([33,255,255])
    threshed_sample = cv2.inRange(hsv, lower_yellow, upper_yellow)
    return threshed_sample

#rock_world = rock_thresh(warped_rock)



def rover_coords(binary_img):
    ypos, xpos = binary_img.nonzero()
    x_pixel = -(ypos - binary_img.shape[0]).astype(np.float)
    y_pixel = -(xpos - binary_img.shape[1]/2 ).astype(np.float)
    return x_pixel, y_pixel

def to_polar_coords(x_pixel, y_pixel):
    dist = np.sqrt(x_pixel**2 + y_pixel**2)
    angles = np.arctan2(y_pixel, x_pixel)
    return dist, angles

def rotate_pix(xpix, ypix, yaw):
    yaw_rad = np.float(yaw) * np.pi / 180
    xpix_rotated = (xpix * np.cos(yaw_rad)) - (ypix * np.sin(yaw_rad))
    ypix_rotated = (xpix * np.sin(yaw_rad)) + (ypix * np.cos(yaw_rad))
    return xpix_rotated, ypix_rotated

def translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale): 
    xpix_translated = (xpix_rot / scale) + xpos
    ypix_translated = (ypix_rot / scale) + ypos
    return xpix_translated, ypix_translated

def pix_to_world(xpix, ypix, xpos, ypos, yaw, world_size, scale):
    xpix_rot, ypix_rot = rotate_pix(xpix, ypix, yaw)
    xpix_tran, ypix_tran = translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale)
    x_pix_world = np.clip(np.int_(xpix_tran), 0, world_size - 1)
    y_pix_world = np.clip(np.int_(ypix_tran), 0, world_size - 1)
    return x_pix_world, y_pix_world



#><><><><><><><>!!!---NEW PART---!!!><><><><><><><><><>

# Import pandas and read in csv file as a dataframe
import pandas as pd
# Change the path below to your data directory
# If you are in a locale (e.g., Europe) that uses ',' as the decimal separator
# change the '.' to ','
df = pd.read_csv('../test_dataset/robot_log.csv', delimiter=';', decimal='.')
csv_img_list = df["Path"].tolist() # Create list of image pathnames
# Read in ground truth map and create a 3-channel image with it
ground_truth = mpimg.imread('../calibration_images/map_bw.png')
ground_truth_3d = np.dstack((ground_truth*0, ground_truth*255, ground_truth*0)).astype(np.float)

# Creating a class to be the data container
# Will read in saved data from csv file and populate this object
# Worldmap is instantiated as 200 x 200 grids corresponding 
# to a 200m x 200m space (same size as the ground truth map: 200 x 200 pixels)
# This encompasses the full range of output position values in x and y from the sim
class Databucket():
    def __init__(self):
        self.images = csv_img_list  
        self.xpos = df["X_Position"].values
        self.ypos = df["Y_Position"].values
        self.yaw = df["Yaw"].values
        self.count = -1 # This will be a running index, setting to -1 is a hack
                        # because moviepy (below) seems to run one extra iteration
        self.worldmap = np.zeros((200, 200, 3)).astype(np.float)
        self.ground_truth = ground_truth_3d # Ground truth worldmap

# Instantiate a Databucket().. this will be a global variable/object
# that you can refer to in the process_image() function below
data = Databucket()


# Define a function to pass stored images to
# reading rover position and yaw angle from csv file
# This function will be used by moviepy to create an output video
def process_image(img):
    # Example of how to use the Databucket() object defined above
    # to print the current x, y and yaw values 
    # print(data.xpos[data.count], data.ypos[data.count], data.yaw[data.count])
    
    #IMAGE DEFINITIONS
    image_imread = mpimg.imread(data.images[data.count])
    image_cv2 = cv2.imread(data.images[data.count])
    image_otsu = cv2.imread(data.images[data.count],0)
    
    # 1) Define source and destination points for perspective transform
    dst_size = 5
    bottom_offset = 6
    source = np.float32([[15, 140], [301 ,140],[200, 96], [118, 96]])
    destination = np.float32([[image_imread.shape[1]/2 - dst_size, image_imread.shape[0] - bottom_offset],
                      [image_imread.shape[1]/2 + dst_size, image_imread.shape[0] - bottom_offset],
                      [image_imread.shape[1]/2 + dst_size, image_imread.shape[0] - 2*dst_size - bottom_offset], 
                      [image_imread.shape[1]/2 - dst_size, image_imread.shape[0] - 2*dst_size - bottom_offset],
                      ])
    
    # 2) Apply perspective transform
    warped = perspect_transform(image_otsu, source, destination)
    warped_rock = perspect_transform(image_cv2, source, destination)
    
    # 3) Apply color threshold to identify navigable terrain/obstacles/rock samples
    navi_world = navigable_thresh(warped)
    obst_world = obstacle_thresh(warped)
    rock_world = rock_thresh(warped_rock)
    
    # 4) Convert thresholded image pixel values to rover-centric coords
    #CONVERT FROM IMAGE COORDS TO ROVER COORDS
    navi_x_pix, navi_y_pix = rover_coords(navi_world)
    obst_x_pix, obst_y_pix = rover_coords(obst_world)
    rock_x_pix, rock_y_pix = rover_coords(rock_world)
    
    #CONVERT TO RADIAL COORDS IN ROVER SPACE
    navi_dist, navi_angles = to_polar_coords(navi_x_pix, navi_y_pix)
    obst_dist, obst_angles = to_polar_coords(obst_x_pix, obst_y_pix)
    rock_dist, rock_angles = to_polar_coords(rock_x_pix, rock_y_pix)
    
    # 5) Convert rover-centric pixel values to world coords 
    #FROM DATABUCKET CLASS
    rover_xpos = data.xpos[data.count]
    rover_ypos = data.ypos[data.count]
    rover_yaw = data.yaw[data.count]
    scale = 10
    shape = data.worldmap.shape[1]
    
    #ROTATE_PIX(X_PIX, Y_PIX, YAW)
    navi_xpix_rot, navi_ypix_rot = rotate_pix(navi_x_pix, navi_y_pix, rover_yaw)
    obst_xpix_rot, obst_ypix_rot = rotate_pix(obst_x_pix, obst_y_pix, rover_yaw)
    rock_xpix_rot, rock_ypix_rot = rotate_pix(rock_x_pix, rock_y_pix, rover_yaw)
    
    #TRANSLATE_PIX(XPIX_ROT, YPIX_ROT, XPOS, YPOS, SCALE)
    navi_xpix_trans, navi_ypix_trans = translate_pix(navi_xpix_rot, navi_ypix_rot, rover_xpos, rover_ypos, scale)
    obst_xpix_trans, obst_ypix_trans = translate_pix(obst_xpix_rot, obst_ypix_rot, rover_xpos, rover_ypos, scale)
    rock_xpix_trans, rock_ypix_trans = translate_pix(rock_xpix_rot, rock_ypix_rot, rover_xpos, rover_ypos, scale)
    
    #PIX_TO_WORLD(XPIX, YPIX, XPOS, YPOS, YAW, WORLD_SIZE, SCALE)
    #ROVER-CENTRIC COORD PIXEL VALUES
    navi_x_world, navi_y_world = pix_to_world(navi_x_pix, navi_y_pix, rover_xpos, rover_ypos, rover_yaw, shape, scale)
    obst_x_world, obst_y_world = pix_to_world(obst_x_pix, obst_y_pix, rover_xpos, rover_ypos, rover_yaw, shape, scale)
    rock_x_world, rock_y_world = pix_to_world(rock_x_pix, rock_y_pix, rover_xpos, rover_ypos, rover_yaw, shape, scale)
    
    #mean_dir = np.mean(angles)
    
    # 6) Update worldmap (to be displayed on right side of screen)
    
    data.worldmap[obst_y_world, obst_x_world, 0] += (255,0,0)
    data.worldmap[rock_y_world, rock_x_world, 1] += (0,255,0)
    data.worldmap[navi_y_world, navi_x_world, 2] += (0,0,255)
    
    # 7) Make a mosaic image, below is some example code
        # First create a blank image (can be whatever shape you like)
    output_image = np.zeros((img.shape[0] + data.worldmap.shape[0], img.shape[1]*2, 3))
        # Next you can populate regions of the image with various output
        # Here I'm putting the original image in the upper left hand corner
    output_image[0:img.shape[0], 0:img.shape[1]] = img

        # Let's create more images to add to the mosaic, first a warped image
    warped = perspect_transform(img, source, destination)
        # Add the warped image in the upper right hand corner
    output_image[0:img.shape[0], img.shape[1]:] = warped

        # Overlay worldmap with ground truth map
    map_add = cv2.addWeighted(data.worldmap, 1, data.ground_truth, 0.5, 0)
        # Flip map overlay so y-axis points upward and add to output_image 
    output_image[img.shape[0]:, 0:data.worldmap.shape[1]] = np.flipud(map_add)


        # Then putting some text over the image
    cv2.putText(output_image,"Populate this image with your analyses to make a video!", (20, 20), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1)
    data.count += 1 # Keep track of the index in the Databucket()
    
    return output_image

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from moviepy.editor import ImageSequenceClip


# Define pathname to save the output video
output = '../output/test_mapping.mp4'
data = Databucket() # Re-initialize data in case you're running this cell multiple times
clip = ImageSequenceClip(data.images, fps=60) # Note: output video will be sped up because 
                                          # recording rate in simulator is fps=25
new_clip = clip.fl_image(process_image) #NOTE: this function expects color images!!
%time new_clip.write_videofile(output, audio=False)


ValueError: operands could not be broadcast together with shapes (48047,) (3,) (48047,) 

In [8]:
scale = str(10)
type (scale)

str